In [ ]:
!apt-get install -y cmake gcc libssl-dev python3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
libssl-dev is already the newest version (3.0.2-0ubuntu1.19).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!git clone --branch main https://github.com/open-quantum-safe/liboqs.git
!git clone --branch main https://github.com/open-quantum-safe/liboqs-python.git

Cloning into 'liboqs'...
remote: Enumerating objects: 37277, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 37277 (delta 139), reused 114 (delta 109), pack-reused 37043 (from 2)
Receiving objects: 100% (37277/37277), 149.58 MiB | 20.14 MiB/s, done.
Resolving deltas: 100% (27245/27245), done.
Cloning into 'liboqs-python'...
remote: Enumerating objects: 735, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 735 (delta 197), reused 160 (delta 154), pack-reused 492 (from 2)
Receiving objects: 100% (735/735), 181.81 KiB | 1.16 MiB/s, done.
Resolving deltas: 100% (369/369), done.


In [ ]:
%cd liboqs
!cmake -DOQS_ENABLE_SIG_FALCON=ON -DOQS_BUILD_ONLY_SIG_FALCON=ON .
!make -j4  # Adjust the number of jobs as needed


/content/liboqs
-- The C compiler identification is GNU 11.4.0
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Performing Test CC_SUPPORTS_WA_NOEXECSTACK
-- Performing Test CC_SUPPORTS_WA_NOEXECSTACK - Success
-- Performing Test LD_SUPPORTS_WL_Z_NOEXECSTACK
-- Performing Test LD_SUPPORTS_WL_Z_NOEXECSTACK - Success
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Alg enablement unchanged
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found suitable version "3.0.2", minimum required is "1.1.1")
-- Looking for aligned_alloc
-- Looking for aligned_alloc - found
-- Looking for posix_memalign
-- Looking for posix_memalign - found
-- Looking for memalign
-- Looking for memalign - found


In [ ]:
%cd ../liboqs-python
!python3 setup.py install

/content/liboqs/liboqs-python
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-b

In [ ]:
import oqs

# Test if Falcon-512 is available
try:
    signature = oqs.Signature("Falcon-512")
    print("Falcon-512 is available.")
except Exception as e:
    print(f"Error: {e}")

liboqs not found, installing it in /root/_oqs
5 4 3 2 1 
Done installing liboqs
Falcon-512 is available.


In [ ]:
!pip install cryptography


In [ ]:
!pip install pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.0 MB/s eta 0:00:00


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.backends import default_backend
from oqs import Signature
import os
from Crypto.Util.Padding import pad, unpad

# Generate a random key for ChaCha20 (32 bytes long)
def generate_random_key():
    return os.urandom(32)

# ChaCha20 Encryption using a generated key
def encrypt_file_chacha20(file_path, key):
    nonce = os.urandom(16)
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
    encryptor = cipher.encryptor()

    # Read file and encrypt
    with open(file_path, "rb") as file:
        file_data = file.read()
    ciphertext = encryptor.update(pad(file_data, 64)) + encryptor.finalize()  # Pad to a multiple of 64 for consistency
    return nonce + ciphertext

# ChaCha20 Decryption for file using generated key
def decrypt_file_chacha20(encrypted_data, key):
    nonce = encrypted_data[:16]
    ciphertext = encrypted_data[16:]

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = unpad(decryptor.update(ciphertext) + decryptor.finalize(), 64)

    return decrypted_data

# Generate Falcon signature for the encrypted data
def sign_file(data):
    sig = Signature("Falcon-512")
    public_key = sig.generate_keypair()
    private_key = sig.export_secret_key()
    signature = sig.sign(data)
    return public_key, private_key, signature

# Verify Falcon signature
def verify_signature(data, signature, public_key):
    sig = Signature("Falcon-512")
    return sig.verify(data, signature, public_key)

# ChaCha20 Encryption with Falcon Signature
def falcon_file_encryption(file_path):
    # Generate a random key
    key = generate_random_key()

    # Encrypt the file
    encrypted_file_data = encrypt_file_chacha20(file_path, key)

    # Generate Falcon signature for the encrypted data
    public_key, private_key, signature = sign_file(encrypted_file_data)

    return encrypted_file_data, key, public_key, signature

# Decryption and Signature Verification
def falcon_file_decryption(encrypted_file_data, key, public_key, signature):
    # Verify the Falcon signature
    if not verify_signature(encrypted_file_data, signature, public_key):
        raise ValueError("Signature verification failed. The file may have been tampered with.")

    # Decrypt the file data
    decrypted_file_data = decrypt_file_chacha20(encrypted_file_data, key)

    return decrypted_file_data

# Main function for encryption and decryption with Falcon signature
if __name__ == "__main__":
    # Input file path
    file_path = input("Enter the path to the file you want to encrypt: ")

    # Perform encryption with Falcon signature
    encrypted_file_data, key, public_key, signature = falcon_file_encryption(file_path)
    print("File encrypted and signed successfully.")

    # Save the encrypted data, and signature
    encrypted_file_path = "encrypted_file"
    with open(encrypted_file_path, "wb") as enc_file:
        enc_file.write(encrypted_file_data)
    print(f"Encrypted data saved to {encrypted_file_path}.")

    # Decrypt and verify the file
    with open(encrypted_file_path, "rb") as enc_file:
        encrypted_file_data = enc_file.read()

    decrypted_file_data = falcon_file_decryption(encrypted_file_data, key, public_key, signature)

    # Save the decrypted file
    decrypted_file_path = "decrypted_file"
    with open(decrypted_file_path, "wb") as dec_file:
        dec_file.write(decrypted_file_data)
    print("File decrypted and signature verified successfully. Saved to decrypted_file.txt.")


Enter the path to the file you want to encrypt: /content/file.txt
File encrypted and signed successfully.
Encrypted data saved to encrypted_file.
File decrypted and signature verified successfully. Saved to decrypted_file.txt.
